In [18]:
for i in {1..3}; do
    d="$PWD/data/mongodb-replSetI-${i}"
    mkdir -p $d/db
    mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI &
done

[1]   Done                    mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI
[2]   Done                    mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI
[3]   Done                    mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI
[4]-  Done                    mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI
[5]+  Done                    mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI
[1] 26496
[2] 26498
[3] 26500


In [19]:
netstat -nltp | grep mongod

(Not all processes could be identified, non-owned process info
 will not be shown, you would have to be root to see it all.)
tcp        0      0 127.0.0.1:2701          0.0.0.0:*               LISTEN      26496/mongod        
tcp        0      0 127.0.0.1:2702          0.0.0.0:*               LISTEN      26498/mongod        
tcp        0      0 127.0.0.1:2703          0.0.0.0:*               LISTEN      26500/mongod        


In [20]:
cat > replSetI_config.js <<EOF
var cfg = {
  '_id':'replSetI',
  'members':[
    {'_id':1, 'host': 'localhost:2701'},
    {'_id':2, 'host': 'localhost:2702'},
    {'_id':3, 'host': 'localhost:2703'},
  ]
};

rs.initiate(cfg);
EOF

mongo --host localhost:2701 < replSetI_config.js

rm replSetI_config.js

MongoDB shell version v3.6.3
connecting to: mongodb://localhost:2701/
MongoDB server version: 3.6.3
{
	"ok" : 1,
	"operationTime" : Timestamp(1603781378, 1),
	"$clusterTime" : {
		"clusterTime" : Timestamp(1603781378, 1),
		"signature" : {
			"hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
			"keyId" : NumberLong(0)
		}
	}
}
bye


In [22]:
master=""
for i in {1..3}; do
    host="localhost:270$i"
    ismaster=$(echo "rs.isMaster()['ismaster']" | mongo --quiet --host $host)

    if [[ "$ismaster" == "true" ]]; then
        master=$host
        echo ">>> master node: "$master
        break
    fi
done

echo "rs.status();" | mongo --host $master

>>> master node: localhost:2701
MongoDB shell version v3.6.3
connecting to: mongodb://localhost:2701/
MongoDB server version: 3.6.3
{
	"set" : "replSetI",
	"date" : ISODate("2020-10-27T06:49:54.279Z"),
	"myState" : 1,
	"term" : NumberLong(1),
	"heartbeatIntervalMillis" : NumberLong(2000),
	"optimes" : {
		"lastCommittedOpTime" : {
			"ts" : Timestamp(1603781390, 5),
			"t" : NumberLong(1)
		},
		"readConcernMajorityOpTime" : {
			"ts" : Timestamp(1603781390, 5),
			"t" : NumberLong(1)
		},
		"appliedOpTime" : {
			"ts" : Timestamp(1603781390, 5),
			"t" : NumberLong(1)
		},
		"durableOpTime" : {
			"ts" : Timestamp(1603781390, 5),
			"t" : NumberLong(1)
		}
	},
	"members" : [
		{
			"_id" : 1,
			"name" : "localhost:2701",
			"health" : 1,
			"state" : 1,
			"stateStr" : "PRIMARY",
			"uptime" : 29,
			"optime" : {
				"ts" : Timestamp(1603781390, 5),
				"t" : NumberLong(1)
			},
			"optimeDate" : ISODate("2020-10-27T06:49:50Z"),
			"infoMessage" : "could not find member to sync from"

In [23]:
i=4
d="$PWD/data/mongodb-replSetI-${i}"
mkdir -p $d/db
mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI &
echo "rs.add(\"localhost:270${i}\")" | mongo --host $master

[4] 26786
MongoDB shell version v3.6.3
connecting to: mongodb://localhost:2701/
MongoDB server version: 3.6.3
{
	"ok" : 1,
	"operationTime" : Timestamp(1603781399, 1),
	"$clusterTime" : {
		"clusterTime" : Timestamp(1603781399, 1),
		"signature" : {
			"hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
			"keyId" : NumberLong(0)
		}
	}
}
bye


In [24]:
i=5
d="$PWD/data/mongodb-replSetI-arb"
mkdir -p $d/db
mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI &
echo "rs.addArb(\"localhost:270${i} \")" | mongo --host $master

echo "rs.status();" | mongo --host $master

[5] 26894
MongoDB shell version v3.6.3
connecting to: mongodb://localhost:2701/
MongoDB server version: 3.6.3
{
	"ok" : 1,
	"operationTime" : Timestamp(1603781404, 1),
	"$clusterTime" : {
		"clusterTime" : Timestamp(1603781404, 1),
		"signature" : {
			"hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
			"keyId" : NumberLong(0)
		}
	}
}
bye
MongoDB shell version v3.6.3
connecting to: mongodb://localhost:2701/
MongoDB server version: 3.6.3
{
	"set" : "replSetI",
	"date" : ISODate("2020-10-27T06:50:05.060Z"),
	"myState" : 1,
	"term" : NumberLong(1),
	"heartbeatIntervalMillis" : NumberLong(2000),
	"optimes" : {
		"lastCommittedOpTime" : {
			"ts" : Timestamp(1603781404, 1),
			"t" : NumberLong(1)
		},
		"readConcernMajorityOpTime" : {
			"ts" : Timestamp(1603781404, 1),
			"t" : NumberLong(1)
		},
		"appliedOpTime" : {
			"ts" : Timestamp(1603781404, 1),
			"t" : NumberLong(1)
		},
		"durableOpTime" : {
			"ts" : Timestamp(1603781404, 1),
			"t" : NumberLong(1)
		}
	},
	"members" : [
		

In [25]:
jobs

[1]   Running                 mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI &
[2]   Running                 mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI &
[3]   Running                 mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI &
[4]-  Running                 mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI &
[5]+  Running                 mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI &


In [26]:
cat > tmp.js << EOF
use replSetI;

db.mycol.insert([
  {
    title: "MongoDB Overview",
    description: "MongoDB is no SQL database",
    by: "tutorials point",
    url: "http://www.tutorialspoint.com",
    tags: ["mongodb", "database", "NoSQL"],
    likes: 100
  },
  {
    title: "NoSQL Database",
    description: "NoSQL database doesn't have tables",
    by: "tutorials point",
    url: "http://www.tutorialspoint.com",
    tags: ["mongodb", "database", "NoSQL"],
    likes: 20,
    comments: [
      {
        user:"user1",
        message: "My first comment",
        dateCreated: new Date(2013,11,10,2,35),
        like: 0
      }
    ]
  }
]);

db.mycol.find().pretty();

db.mycol.drop();
EOF

mongo --host $master < tmp.js
rm tmp.js

MongoDB shell version v3.6.3
connecting to: mongodb://localhost:2701/
MongoDB server version: 3.6.3
switched to db replSetI
BulkWriteResult({
	"writeErrors" : [ ],
	"writeConcernErrors" : [ ],
	"nInserted" : 2,
	"nUpserted" : 0,
	"nMatched" : 0,
	"nModified" : 0,
	"nRemoved" : 0,
	"upserted" : [ ]
})
{
	"_id" : ObjectId("5f97c32e20a3f32dd5f706a6"),
	"title" : "MongoDB Overview",
	"description" : "MongoDB is no SQL database",
	"by" : "tutorials point",
	"url" : "http://www.tutorialspoint.com",
	"tags" : [
		"mongodb",
		"database",
		"NoSQL"
	],
	"likes" : 100
}
{
	"_id" : ObjectId("5f97c32e20a3f32dd5f706a7"),
	"title" : "NoSQL Database",
	"description" : "NoSQL database doesn't have tables",
	"by" : "tutorials point",
	"url" : "http://www.tutorialspoint.com",
	"tags" : [
		"mongodb",
		"database",
		"NoSQL"
	],
	"likes" : 20,
	"comments" : [
		{
			"user" : "user1",
			"message" : "My first comment",
			"dateCreated" : ISODate("2013-12-09T18:35:00Z"),
			"like" : 0
		}
	]
}
true
bye


In [28]:
kill %1

In [29]:
jobs

[1]   Done                    mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI
[2]   Running                 mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI &
[3]   Running                 mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI &
[4]-  Running                 mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI &
[5]+  Running                 mongod --port 270${i} --dbpath="$d/db" -logpath="$d/mongo.log" --replSet replSetI &


In [32]:
# kill %{1..5}